# Adversarial training
The model with adversarial training was trained with images containing Gaussian noise and achieved an accuracy of 0.79 on the standard test set (without noise). The normal model, trained with standard images without random noise, performed better with an accuracy of 0.83 on the same test set.

On a training set with added random Gaussian noise, the adversarially trained model outperformed the normally trained model, achieving an accuracy of 0.8138 compared to the normal model's accuracy of 0.78.

This demonstrates that incorporating noise into the training set enhances a model's robustness and resistance to disruptions.

#### Results of adversarial network on images without adversarial

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels1x1, out_channels3x3_reduce, out_channels3x3, out_channels5x5_reduce, out_channels5x5, out_channels_pool):
        super(InceptionModule, self).__init__()

        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels1x1, kernel_size=1),
            nn.BatchNorm2d(out_channels1x1),
            nn.ReLU()
        )

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels3x3_reduce, kernel_size=1),
            nn.BatchNorm2d(out_channels3x3_reduce),
            nn.ReLU(),
            nn.Conv2d(out_channels3x3_reduce, out_channels3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels3x3),
            nn.ReLU()
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels5x5_reduce, kernel_size=1),
            nn.BatchNorm2d(out_channels5x5_reduce),
            nn.ReLU(),
            nn.Conv2d(out_channels5x5_reduce, out_channels5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(out_channels5x5),
            nn.ReLU()
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_channels_pool, kernel_size=1),
            nn.BatchNorm2d(out_channels_pool),
            nn.ReLU()
        )

    def forward(self, x):
        branch1_out = self.branch1(x)
        branch2_out = self.branch2(x)
        branch3_out = self.branch3(x)
        branch4_out = self.branch4(x)

        outputs = [branch1_out, branch2_out, branch3_out, branch4_out]
        return torch.cat(outputs, 1)

class Cifar10Classifier(nn.Module):
    def __init__(self):
        super(Cifar10Classifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.inception1 = InceptionModule(32, 16, 16, 32, 4, 8, 8)
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.inception2 = InceptionModule(64, 32, 32, 64, 8, 16, 16)
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Linear(512, 10)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x: torch.Tensor):
        single_input = False
        if x.ndim == 3: 
            x = x.unsqueeze(dim=0) 
            single_input = True

        x = self.conv1(x)
        x = self.inception1(x)
        x = self.conv2(x)
        x = self.inception2(x)
        x = self.conv3(x)
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.log_softmax(x)

        if single_input:
            x = x.squeeze(dim=0) 

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelA = Cifar10Classifier().to(device)
print(modelA)


Cifar10Classifier(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (inception1): InceptionModule(
    (branch1): Sequential(
      (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (branch2): Sequential(
      (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (branch3): Sequential(
      (0): Conv2d(32, 4, kernel_size=(1

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

all_labels = []
all_preds = []
modelA.load_state_dict(torch.load("/model/modelA_49.pth"))
modelA.eval()

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = modelA(batch_X)
        max_pred_values, preds = torch.max(outputs, 1)
        all_labels.extend(batch_y.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')

results_dict["Adversarial_training"] = (accuracy, precision, recall, f1, conf_matrix) 


Accuracy: 0.7917
Precision: 0.8036
Recall: 0.7917
F1 Score: 0.7926
Confusion Matrix:
[[897  13  13  13   4   1   1   6  28  24]
 [ 12 933   3   3   0   3   0   0   8  38]
 [101   4 698  60  36  53  17  21   4   6]
 [ 39   3  50 693  22 136  19  20  10   8]
 [ 58   2  59  98 614  61  10  89   5   4]
 [ 16   2  35 168  13 724   5  29   4   4]
 [ 11   7  47 105   7  48 760   4   7   4]
 [ 17   2  24  30   8  39   0 870   1   9]
 [ 74  18   5  13   1   4   0   2 871  12]
 [ 29  68   3  13   0   4   1   9  16 857]]


# Comparing model with adversarial training

In [64]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size(), device=tensor.device) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

modelN = Cifar10Classifier().to(device)
modelN.load_state_dict(torch.load("/model/model_49.pth"))
modelA = Cifar10Classifier().to(device)
modelA.load_state_dict(torch.load("/model/modelA_49.pth"))
all_labels = []
all_preds_A = []
all_preds_N = []
modelA.eval()
modelN.eval()
add_noise = AddGaussianNoise(0., 0.05)

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        batch_X = add_noise(batch_X)
        outputs = modelA(batch_X)
        max_pred_values, preds = torch.max(outputs, 1)
        all_labels.extend(batch_y.cpu().numpy())
        all_preds_A.extend(preds.cpu().numpy())
        outputs = modelN(batch_X)
        max_pred_values, preds = torch.max(outputs, 1)
        all_preds_N.extend(preds.cpu().numpy())
        
all_labels = np.array(all_labels)
all_preds_A = np.array(all_preds_A)
all_preds_N = np.array(all_preds_N)

conf_matrix = confusion_matrix(all_labels, all_preds_A)
accuracy = accuracy_score(all_labels, all_preds_A)
precision = precision_score(all_labels, all_preds_A, average='macro')
recall = recall_score(all_labels, all_preds_A, average='macro')
f1 = f1_score(all_labels, all_preds_A, average='macro')

print("Model with adversarial training:")
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')

conf_matrix = confusion_matrix(all_labels, all_preds_N)
accuracy = accuracy_score(all_labels, all_preds_N)
precision = precision_score(all_labels, all_preds_N, average='macro')
recall = recall_score(all_labels, all_preds_N, average='macro')
f1 = f1_score(all_labels, all_preds_N, average='macro')
print("Normal model:")
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Model with adversarial training:
Accuracy: 0.8138
Precision: 0.8132
Recall: 0.8138
F1 Score: 0.8134
Confusion Matrix:
[[838  12  26  16  12   4   3  11  52  26]
 [ 10 923   2   2   0   3   0   1  10  49]
 [ 49   5 714  42  67  53  41  18   7   4]
 [ 17   4  51 654  45 145  37  25  10  12]
 [ 21   3  39  33 796  33  18  49   5   3]
 [ 10   2  40 141  34 715  17  30   3   8]
 [  6   3  32  57  14  22 851   6   6   3]
 [ 10   1  23  22  30  38   0 866   0  10]
 [ 49  10   8   5   3   3   3   4 896  19]
 [ 23  53   6   6   0   3   2   5  17 885]]
Normal model:
Accuracy: 0.7870
Precision: 0.8032
Recall: 0.7870
F1 Score: 0.7869
Confusion Matrix:
[[716   7  60  24  81   2  16   8  54  32]
 [  5 877   4   4   6   1   8   1  15  79]
 [ 23   1 679  25 133  36  88   3   4   8]
 [  5   3  40 565 130  98 121  17   7  14]
 [  1   1  33   9 874   6  60  13   3   0]
 [  3   1  29 107 113 653  61  19   3  11]
 [  0   3  11  11  27  12 933   1   0   2]
 [  4   0  34  27 106  26  15 775   1  12]
 [ 19  1